# Data Quality Visualizations

In [1]:
import time
import boto3
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import contextily as cx

# Set AWS credentials
s3 = boto3.resource("s3")
s3_cl = boto3.client("s3")  # for lower-level processes

# Set relative paths to other folders and objects in repository.
BUCKET_NAME = "wecc-historical-wx"
QAQC_DIR = "3_qaqc_wx"
MERGE_DIR = "4_merge_wx"
stations_csv_path = f"s3://{BUCKET_NAME}/{QAQC_DIR}/all_network_stationlist_qaqc.csv"
shapepath = "s3://wecc-historical-wx/0_maps/tl_2021_us_state"

In [ ]:
station_list = pd.read_csv(stations_csv_path)

In [ ]:
total_flag_path = f"s3://{BUCKET_NAME}/{MERGE_DIR}/total_flag_counts_native_timestep.csv"
total_flag_list = pd.read_csv(total_flag_path)

Append, onto station_list, target variable flag rate.
We want the flag rate per variable per station. Let's test with ASOSAWOS.

In [2]:
network_flag_path = f"s3://{BUCKET_NAME}/{MERGE_DIR}/ASOSAWOS/eraqc_counts_native_timestep/ASOSAWOS_A0705300346_flag_counts_native_timestep.csv"
flag_df_1 = pd.read_csv(network_flag_path)

In [ ]:
network_flag_path_2 = f"s3://{BUCKET_NAME}/{MERGE_DIR}/ASOSAWOS/eraqc_counts_native_timestep/ASOSAWOS_A0705300346_flag_counts_native_timestep.csv"
flag_df_2 = pd.read_csv(network_flag_path)

In [73]:
flag_df_1 = flag_df_1.set_index("eraqc_flag_values")
subset = flag_df_1[~flag_df_1.index.isin(["no_flag", "total_obs_count"])]

totals = subset.sum(numeric_only=True)
flag_df_1.loc["total_flag"] = pd.Series(totals)

frac = flag_df_1.loc["total_flag"] / flag_df_1.loc["total_obs_count"]
flag_df_1.loc["frac"] = pd.Series(frac)

rates_df = flag_df_1.loc[["frac"]]
rates_df = rates_df.rename(index={"frac": 'station_name'})

In [ ]:
flag_df_1 = flag_df_1.set_index("eraqc_flag_values")
subset = flag_df_1[~flag_df_1.index.isin(["no_flag", "total_obs_count"])]

totals = subset.sum(numeric_only=True)
flag_df_1.loc["total_flag"] = pd.Series(totals)

frac = flag_df_1.loc["total_flag"] / flag_df_1.loc["total_obs_count"]
flag_df_1.loc["frac"] = pd.Series(frac)

rates_df = flag_df_1.loc[["frac"]]
rates_df = rates_df.rename(index={"frac": "station_name"})

In [ ]:
def _pairwise_rate(flag_df_1: pd.DataFrame, flag_df_2: pd.DataFrame,station_name: str) -> pd.DataFrame:
    """
    Sums two input flag count dataframes. This is a helper function for sum_flag_counts().

    Parameters
    ----------
    flag_df_1: pd.DataFrame
        dataframe of previously summed station flag counts
    flag_df_2: pd.DataFrame
        flag counts dataframes for next station

    Returns
    -------
    summed_df: pd.DataFrame

    """
    if len(flag_df_2) == 0:
        return flag_df_1
    else:
        flag_df_1 = flag_df_1.set_index("eraqc_flag_values")
        subset = flag_df_1[~flag_df_1.index.isin(["no_flag", "total_obs_count"])]

        totals = subset.sum(numeric_only=True)
        flag_df_1.loc["total_flag"] = pd.Series(totals)

        frac = flag_df_1.loc["total_flag"] / flag_df_1.loc["total_obs_count"]
        flag_df_1.loc["frac"] = pd.Series(frac)
        
        rates_df = flag_df_1.loc[["frac"]]
        rates_df = rates_df.rename(index={"frac": station_name})

        # now append to flag_df_2
        print('here they are')
        rates_df = pd.concat([rates_df, flag_df_2])

        return rates_df

In [8]:
network = "VCAPCD"

In [9]:
# the function iteratively adds in flag counts to this dataframe
flag_rate_df = []

# point to folder containing station flag count CSVs
flags_prefix = f"{MERGE_DIR}/{network}/eraqc_counts_native_timestep"

## Merge flag counts

# loop through all CSVs are the given level
for item in s3.Bucket(BUCKET_NAME).objects.filter(Prefix=flags_prefix):
    obj = s3_cl.get_object(Bucket=BUCKET_NAME, Key=item.key)
    flags = pd.read_csv(obj["Body"])
    station_name = item.key.split(flags_prefix + "/")[1]
    # the CSV is empty
    if flags.empty:
        continue
    # the CSV is not empty
    else:
        # send current dataframe and dataframe of previously summed counts to helper function
        flag_rate_df = _pairwise_rate(flags, flag_rate_df,station_name)
        # print(station_name)

here they are
                                           elevation      hurs     pr_1h   pr  \
eraqc_flag_values                                                               
VCAPCD_OJ_flag_counts_native_timestep.csv        0.0  0.000908  0.000232  0.0   

                                               rsds  sfcWind_dir   sfcWind  \
eraqc_flag_values                                                            
VCAPCD_OJ_flag_counts_native_timestep.csv  0.008078     0.001314  0.000908   

                                                tas  tdps_derived  
eraqc_flag_values                                                  
VCAPCD_OJ_flag_counts_native_timestep.csv  0.000502      0.003614  
  eraqc_flag_values  elevation    hurs   pr_1h      pr      ps    rsds  \
0           no_flag     103040  102868  103040  103040  103035  102976   
1              11.0          0     172       0       0       2      64   
2              23.0          0       0       0       0       3       0   
3      

In [10]:
flag_rate_df

,elevation,hurs,pr_1h,rsds,sfcWind_dir,sfcWind,tas,tdps_derived,ps,pr,eraqc_flag_values
VCAPCD_TO_flag_counts_native_timestep.csv,0.0,0.001498,0.000279,0.001347,0.002822,0.001486,0.003403,0.009395,NaN,NaN,NaN
VCAPCD_SV_flag_counts_native_timestep.csv,0.0,0.001683,0.000000,0.000856,0.002733,0.001537,0.001780,0.007226,0.000311,NaN,NaN
VCAPCD_SU_flag_counts_native_timestep.csv,0.0,0.002337,0.000000,0.002046,0.003850,0.002308,0.004024,0.009144,0.001435,0.0,NaN
VCAPCD_PU_flag_counts_native_timestep.csv,0.0,0.001378,0.000000,0.001227,0.001644,0.001355,0.000706,0.006322,NaN,NaN,NaN
VCAPCD_OJ_flag_counts_native_timestep.csv,0.0,0.000908,0.000232,0.008078,0.001314,0.000908,0.000502,0.003614,NaN,0.0,NaN
0,103040.0,102868.000000,103040.000000,102976.000000,102394.000000,102961.000000,102806.000000,102332.000000,103035.000000,103040.0,no_flag
1,0.0,172.000000,0.000000,64.000000,79.000000,79.000000,91.000000,89.000000,2.000000,0.0,11.0
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,40.000000,3.000000,0.0,23.0
3,0.0,0.000000,0.000000,0.000000,567.000000,0.000000,0.000000,0.000000,0.000000,0.0,15.0
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,102.000000,451.000000,0.000000,0.0,26.0


## Map

In [ ]:
# Format dates in datetime format (this gets lost in import).
station_list["start-date"] = pd.to_datetime(station_list["start-date"], utc=True)
station_list["end-date"] = pd.to_datetime(station_list["end-date"], utc=True)

# Make a geodataframe.
gdf = gpd.GeoDataFrame(
    station_list,
    geometry=gpd.points_from_xy(station_list.longitude, station_list.latitude),
)
gdf.set_crs(epsg=4326, inplace=True)  # Set CRS

# Project data to match base tiles.
gdf_wm = gdf.to_crs(epsg=3857)  # Web mercator

# Read in geometry of continental US.
us = gpd.read_file(shapepath)

# Remove territories, AK, HI
rem_list = ["HI", "AK", "MP", "GU", "AS", "PR", "VI"]
us = us.loc[us.STUSPS.isin(rem_list) == False]

# Use to clip stations
us = us.to_crs(epsg=3857)
gdf_us = gdf_wm.clip(us)

# Plot
ax = gdf_us.plot(
    "network",
    figsize=(15, 15),
    alpha=1,
    markersize=3,
    legend=True,
    cmap="nipy_spectral",
)
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
ax.set_axis_off()

